# ISAF (إسعاف)  
## Integrated Security Assessments for your dev Flow
```
    EeeiiiiiEEiiiii.....                                             
       \|/                                                           
        n______     .....iiiiiEEiiiieeEE                             
       :~;     :                  \|/                                
-----;``~'  +  ;------------ ______n --------------------------------
     `-@-----@-=            :     :~:                                
=========================== ;  +  '~``; =============================
                            =-@-----@-'                              
jgs------------------------------------------------------------------
                                                                     
                   DEVSECOPS IN A PYTHON NUTSHELL        
```

# Purpose
This project is a "simple" python implementation of the DevSecOps Methodology, boiled down to the following picture:
[![DevSecOps](DevSecOps.png)](https://www.gartner.com/doc/1896617/devopssec-creating-agile-triangle)

and an implementation attempt:  

[![Go with the Flow Morty!!](Flow.png)](https://medium.com/@H.A.T/how-to-implement-webs-hospital-b0d8b85389ce)

# Stack Deployement
You can quickly start this PoC (based on OpenFaaS) on Docker Swarm online using the community-run Docker playground: play-with-docker.com (PWD) by clicking the button below:  

[![Try in PWD](https://cdn.rawgit.com/play-with-docker/stacks/cff22438/assets/images/button.png)](http://labs.play-with-docker.com/?stack=https://gist.githubusercontent.com/h-a-t/eafbb19d7ce46c4ee4a541df018a5f37/raw/d1aa99f6fec38620f09ecd3a9bbfb79207cf3dde/docker-compose.yml&stack_name=func)

Or use the docker-compose.yml file.

# Prerequisite
- Get your Jupyter token:

```bash
$ docker service logs func_jupyter 2>&1 | grep token
func_jupyter.1.xam77gaqxi5s@node1    | [I 18:42:54.562 LabApp] The Jupyter Notebook is running at: http://[all ip addresses on your system]:8888/?token=5afdefee1c98acac0bbf29ae9972b7ccd23c50c115e74e49
func_jupyter.1.xam77gaqxi5s@node1    |     to login with a token:
func_jupyter.1.xam77gaqxi5s@node1    |         http://localhost:8888/?token=5afdefee1c98acac0bbf29ae9972b7ccd23c50c115e74e49
```

- Install git, unzip, curl and faas-cli in the Jupyter container:

```bash
docker exec --user root -ti func_jupyter.1.shw9s15u6co3cuzp5sjft697t bash
root@b9300915e6ad:~# apt-get update && apt-get -y install unzip git curl
root@b9300915e6ad:~# curl -sSL https://cli.openfaas.com | sh # Not cool :/
```

- Change permissions of docker.sock for the sack of this PoC. :warning: Do not do this in a production environment :bomb:

```bash
chmod 777 /var/run/docker.sock
```

- Upload ISAF.iynb to your Jupyter instance, and press play! \o/


# PoC Stack built upon  

| Name       | Link                                       | License                   |
|------------|--------------------------------------------|---------------------------|
|    Clair   | https://github.com/coreos/clair            |        Apache License 2.0 |
| JupyterLab | https://github.com/jupyterlab/jupyterlab   |              BSD 3-Clause |
|    Klar    | https://github.com/optiopay/klar           |                       MIT |
|    Nmap    | https://github.com/nmap/nmap               |        GNU General Public |
|  OpenFaaS  | https://github.com/openfaas/faas           |                       MIT |
|  Sonarqube | https://github.com/SonarSource/sonarqube/  | GNU Lesser General Public |
|   WhatWeb  | https://github.com/urbanadventurer/WhatWeb |                     GPLv2 |
|   WPScan   | https://github.com/wpscanteam/wpscan       |             Dual-Licensed |



# Build

### What App?
A tiny vulnerable application written in PHP.
[![Vulnerable App](screen_webapp.png)](https://github.com/h-a-t/RedPill/blob/master/src/php/index.php)    
These 2 lines are vulnerable to SQL injections and to XSS attacks. 
```php
    $user_id = $_GET['id'];
    $sql = mysql_query("SELECT username, nom, prenom, email FROM users WHERE user_id = $user_id") or die(mysql_error());
```

### Let's deploy it!

In [1]:
!cd ~ && git clone https://github.com/h-a-t/RedPill

Cloning into 'RedPill'...
remote: Counting objects: 168, done.
remote: Total 168 (delta 0), reused 0 (delta 0), pack-reused 168
Receiving objects: 100% (168/168), 11.10 MiB | 0 bytes/s, done.
Resolving deltas: 100% (78/78), done.
Checking connectivity... done.


In [2]:
!pip install docker

    100% |████████████████████████████████| 112kB 3.2MB/s 
    100% |████████████████████████████████| 204kB 2.9MB/s 


In [3]:
import docker
import io
import tarfile
import os
import time

cli = docker.from_env()
cli.containers.list()

[<Container: 3c25ecffac>,
 <Container: 1b24a6f77f>,
 <Container: 32e9d9af41>,
 <Container: 0fcc228201>,
 <Container: a3e6d1cae0>,
 <Container: 6ecd2d1337>,
 <Container: c376d25e98>]

In [6]:
## Build app image and pull dependencies
## May take a fair amount of time

os.chdir(os.path.expanduser('~/RedPill/'))
cli.images.build(path='./src/php', tag='hat/app')
#zap_img = cli.images.pull('owasp/zap2docker-weekly:latest')
db_img = cli.images.pull('mariadb:latest')
alpine = cli.images.pull('alpine:latest')
cli.images.list()

[<Image: 'hat/app:latest'>,
 <Image: 'registry:2'>,
 <Image: 'alpine:latest'>,
 <Image: 'mariadb:latest'>,
 <Image: 'sonarqube:latest'>,
 <Image: 'jupyter/base-notebook:latest'>,
 <Image: 'php:5-apache'>,
 <Image: 'functions/gateway:0.6.6-beta'>,
 <Image: 'functions/prometheus:latest'>,
 <Image: 'functions/alertmanager:latest'>,
 <Image: 'functions/webhookstash:latest'>]

In [7]:
## Create a dedicated network
cli.networks.create("app_net", driver="overlay")
for x in cli.networks.list():
    print("%s  %s" % (x.id,x.name))

85az7z9559821o4kfm4qw25wu  ingress
siex3a4qqdy7u4a1iu5s77v3w  func_functions
wot94unzoim96wyh74tfxmvao  app_net
2136b4fb6766c661b7631b852cf7e41240513417770f0e785dfb02c591c8ea8b  docker_gwbridge
66271f878ad9e144ebb83a182e0b0097a3a6be531dc8c16d17c05815b8e7fc36  none
352142ea0e5f855f1da4bf1a0311e13a4244d5b7bfc89164d2f5773d63ac67e8  host
5572431e57063f6382e23dc68f89e68d7c2d8569b5753688913fc7286641e6a2  bridge


In [8]:
## Create Volumes
cli.volumes.create(name='db_data', driver='local')
cli.volumes.create(name='db_init', driver='local')
cli.volumes.create(name='app_data', driver='local')

<Volume: app_data>

In [11]:
## Database provisionning
os.chdir(os.path.expanduser('~/RedPill/src/sql'))
tarstream = io.BytesIO()
tar = tarfile.TarFile(fileobj=tarstream, mode='w')
tar.add('staging.sql')
tar.close()

## https://gist.github.com/zbyte64/6800eae10ce082bb78f0b7a2cca5cbc2
tmp=cli.containers.create(
    image='alpine', 
    volumes={'db_init':{'bind': '/data/', 'mode' : 'rw'}})

tarstream.seek(0)
tmp.put_archive(
    path='/data/',
    data=tarstream
)

## Database run
db_cont = cli.services.create(
    image='mariadb:latest',
    mounts=[
        "db_init:/docker-entrypoint-initdb.d/:rw",
        "db_data:/var/lib/mysql/:rw"
    ],
    networks= ['app_net'],
    name='db',
    env=['MYSQL_RANDOM_ROOT_PASSWORD=yes','MYSQL_USER=user',
                  'MYSQL_PASSWORD=password','MYSQL_DATABASE=sqli']
        )
db_cont?

Type:        Service
String form: <Service: t16ldwu5fh>
File:        /opt/conda/lib/python3.6/site-packages/docker/models/services.py
Docstring:   A service.


In [13]:
## Webserver provisionning

os.chdir(os.path.expanduser('~/RedPill/src/php'))
tarstream = io.BytesIO()
tar = tarfile.TarFile(fileobj=tarstream, mode='w')
tar.add('.')
tar.close()

tmp=cli.containers.create(
    image='alpine', 
    volumes={'app_data':{'bind': '/data/', 'mode' : 'rw'}})

tarstream.seek(0)
tmp.put_archive(
    path='/data/',
    data=tarstream
)
## webserver run
app_cont = cli.services.create(
    image='hat/app',
    mounts=[
        "app_data:/var/www/html:rw",
    ],
    networks= ['app_net','func_functions'],
    name='app_web',
    endpoint_spec={
                'Mode': 'vip',
                "Ports": [
                    {
                        "Protocol": "tcp",
                        "TargetPort": 80,
                        "PublishedPort": 80
                    }]
                    },
    env=['DB_ENV_MYSQL_USER=user','DB_ENV_MYSQL_PASSWORD=password','BUILD_STAGE=Python'],
        )
app_cont?

Type:        Service
String form: <Service: s59q11g9ir>
File:        /opt/conda/lib/python3.6/site-packages/docker/models/services.py
Docstring:   A service.


# Security Assessment in a Synchronous Execution Flow

## Static Code Analysis


### Push code to SonarQube for code analysis

In [14]:
## Download Sonarqube scanner
os.chdir(os.path.expanduser('~/RedPill'))
!wget https://sonarsource.bintray.com/Distribution/sonar-scanner-cli/sonar-scanner-cli-3.0.3.778-linux.zip
!unzip sonar-scanner-cli-3.0.3.778-linux.zip

--2017-10-30 16:16:02--  https://sonarsource.bintray.com/Distribution/sonar-scanner-cli/sonar-scanner-cli-3.0.3.778-linux.zip
Resolving sonarsource.bintray.com (sonarsource.bintray.com)... 108.168.243.150, 108.168.243.150
Connecting to sonarsource.bintray.com (sonarsource.bintray.com)|108.168.243.150|:443... connected.
HTTP request sent, awaiting response... 302 
Location: https://akamai.bintray.com/c8/c84682a85df0f7f48bd5c14baff6e316603b521a5a10519184ef2f2f009b8623?__gda__=exp=1509380882~hmac=d693df97d3145e74ca40ce483d96405f6f4dd652161aefbc418bba1ae244f419&response-content-disposition=attachment%3Bfilename%3D%22sonar-scanner-cli-3.0.3.778-linux.zip%22&response-content-type=application%2Fzip&requestInfo=U2FsdGVkX19R_lpLomaQU_13mLTmAkyOU9GWsXr8TtIFZau71sebFPmnsw33K21bi6ikPhv4G4ZVryNCIYzCSmxrL92506NI_5Ds3kHhps5dKHcAK-E9GW1-4Q4yAvdyEbVxcaAc-XuUewHwLv-fvoNmTZbEoJRmuPu7Pbjnyng&response-X-Checksum-Sha1=8de2c77c6e5fcf2f939b014397e9e0ec21c18075&response-X-Checksum-Sha2=c84682a85df0f7f48bd5c14b

In [22]:
!ls ./isaf/RedPill/src/php
!./sonar-scanner-3.0.3.778-linux/bin/sonar-scanner -Dsonar.host.url=http://sonarqube:9000 -Dsonar.projectKey=Redpill:latest -Dsonar.sources=~/RedPill/src/php -Dsonar.language=php

debian.png  docker.png	jenkins.png  owasp.png	   star.png
Dockerfile  index.php	lamp.jpg     scaleway.svg  traefik.png
INFO: Scanner configuration file: /home/jovyan/sonar-scanner-3.0.3.778-linux/conf/sonar-scanner.properties
INFO: Project root configuration file: NONE
INFO: SonarQube Scanner 3.0.3.778
INFO: Java 1.8.0_121 Oracle Corporation (64-bit)
INFO: Linux 4.4.0-1035-aws amd64
INFO: User cache: /home/jovyan/.sonar/cache
INFO: Publish mode
INFO: Load global settings
INFO: Load global settings (done) | time=89ms
INFO: User cache: /home/jovyan/.sonar/cache
INFO: Load plugins index
INFO: Load plugins index (done) | time=62ms
INFO: SonarQube server 6.6.0
INFO: Default locale: "en_US", source code encoding: "UTF-8" (analysis is platform dependent)
INFO: Process project properties
INFO: Load project repositories
INFO: Load project repositories (done) | time=18ms
INFO: Load quality profiles
INFO: Load quality profiles (done) | time=76ms
INFO: Load active rules
INFO: Load active rules (do

### Push to Clair for a container layer scan

TODO: Texte à broder

In [23]:
os.chdir(os.path.expanduser('~/work'))
!wget -O clair_conf.yml https://raw.githubusercontent.com/coreos/clair/master/config.yaml.sample
# Database connection string
!sed -i "s/host=.*/postgresql:\/\/postgres:password@db_clair:5432?sslmode=disable/" clair_conf.yml

--2017-10-30 16:48:56--  https://raw.githubusercontent.com/coreos/clair/master/config.yaml.sample
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3141 (3.1K) [text/plain]
Saving to: ‘clair_conf.yml’

clair_conf.yml      100%[===================>]   3.07K  --.-KB/s    in 0s      

2017-10-30 16:48:56 (58.6 MB/s) - ‘clair_conf.yml’ saved [3141/3141]



In [39]:
## Database run
db_clair = cli.services.create(
    image='postgres:latest',
    networks= ['app_net'],
    name='db_clair',
    env=['POSTGRES_PASSWORD=password']
        )
db_clair?


Type:        Service
String form: <Service: z56rew6lx4>
File:        /opt/conda/lib/python3.6/site-packages/docker/models/services.py
Docstring:   A service.


In [40]:
## Config Provisionning to Clair

tarstream = io.BytesIO()
tar = tarfile.TarFile(fileobj=tarstream, mode='w')
tar.add('clair_conf.yml')
tar.close()

## https://gist.github.com/zbyte64/6800eae10ce082bb78f0b7a2cca5cbc2
tmp=cli.containers.create(
    image='alpine', 
    volumes={'clair_init':{'bind': '/data/', 'mode' : 'rw'}})

tarstream.seek(0)
tmp.put_archive(
    path='/data/',
    data=tarstream
)

## Clair run
app_clair = cli.services.create(
    image='quay.io/coreos/clair:latest',
    mounts=[
        "clair_init:/config/:rw",
    ],
    networks= ['app_net','func_functions'],
    name='app_clair',
    endpoint_spec={
                'Mode': 'vip',
                "Ports": [
                    {
                        "Protocol": "tcp",
                        "TargetPort": 6060,
                        "PublishedPort": 6060
                    },
                    {
                        "Protocol": "tcp",
                        "TargetPort": 6061,
                        "PublishedPort": 6061                        
                    }
                ]                
    },
    command=['/clair','-config','/config/clair_conf.yml']        
)
app_clair?


Type:        Service
String form: <Service: bg1z0qfjq7>
File:        /opt/conda/lib/python3.6/site-packages/docker/models/services.py
Docstring:   A service.


In [41]:
## Klar dans un container, parce que flemme
## https://github.com/optiopay/klar/releases
Dockerfile ='''
FROM alpine:latest
RUN apk update && apk add ca-certificates
ADD https://github.com/optiopay/klar/releases/download/v1.5-RC2/klar-1.5-RC2-linux-amd64 /klar
RUN chmod +x /klar
ENV CLAIR_ADDR=http://localhost:6060
ENV CLAIR_OUTPUT=High
ENV CLAIR_THRESHOLD=10
CMD ["/klar"]
'''
#write the Dockerfile to a tarred archive
pw_tarstream = io.BytesIO()
pw_tar = tarfile.TarFile(fileobj=pw_tarstream, mode='w')
file_data = Dockerfile.encode('utf8')
tarinfo = tarfile.TarInfo(name='Dockerfile')
tarinfo.size = len(file_data)
tarinfo.mtime = time.time()
#tarinfo.mode = 0600
pw_tar.addfile(tarinfo, io.BytesIO(file_data))
pw_tar.close()

pw_tarstream.seek(0)
klar = cli.images.build(
    fileobj=pw_tarstream,
    custom_context=True,
    tag='hat/klar'
)
klar?

Type:        Image
String form: <Image: 'hat/klar:latest'>
File:        /opt/conda/lib/python3.6/site-packages/docker/models/images.py
Docstring:   An image on the server.


In [42]:
app_klar = cli.services.create(
    image='hat/klar',
    networks= ['app_net'],
    name='app_klar',
    command=['/klar', 'postgres'],
    env=['CLAIR_ADDR=http://app_clair:6060']
        )
app_klar?
## TODO : Gerer le push sur les registres locaux, plus voir cette histoire de
## failed to load system roots and no roots provided


Type:        Service
String form: <Service: tglmi672gi>
File:        /opt/conda/lib/python3.6/site-packages/docker/models/services.py
Docstring:   A service.


## Dynamic Runtime Analysis

### Using the Web-GUI of your favorite Pentesting tool from OWASP: ZAP
- Let L33t do the testing by running a GUI instance of ZAP, just browse localhost:8666/?anonym=true&app=ZAP to start :) 
- Warning, image size: 1,52 Go  
- Cf. https://github.com/zaproxy/zaproxy/wiki/WebSwing  

In [43]:
zap_img = cli.images.pull('owasp/zap2docker-stable:latest')
scan_cont = cli.containers.run(
    image=zap_img,
    name='app_scan',
    detach=True,
    command="sh -c 'zap-webswing.sh'",
    ports={'8080/tcp':8666, '8090/tcp':8777},
    links=[('app_web','app')]
        )
scan_cont?

NotFound: 404 Client Error: Not Found ("could not get container for app_web: No such container: app_web")

### Using the flexibility of your favorite cutting-edge technology: OpenFaas

Dockerfile:  

```
FROM alexellis2/faas-alpinefunction:latest
RUN apk update && apk add nmap
ENV fprocess="xargs nmap"
CMD ["fwatchdog"]
```

nmap_stack:   

```yaml
provider:
  name: faas
  gateway: http://gateway:8080

functions:
  nmap:
    lang: Dockerfile
    handler: ./Dockerfile
    image: hat/nmap
```

In [ ]:
import tarfile
import time
from io import BytesIO
os.chdir('/home/jovyan/work')
Dockerfile ='''
FROM alexellis2/faas-alpinefunction:latest
RUN apk update && apk add nmap
ENV fprocess="xargs nmap"
CMD ["fwatchdog"]
'''
with open("Dockerfile", "w") as stack:
    stack.write("%s" % Dockerfile)

#write the Dockerfile to a tarred archive
pw_tarstream = BytesIO()
pw_tar = tarfile.TarFile(fileobj=pw_tarstream, mode='w')
file_data = Dockerfile.encode('utf8')
tarinfo = tarfile.TarInfo(name='Dockerfile')
tarinfo.size = len(file_data)
tarinfo.mtime = time.time()
#tarinfo.mode = 0600
pw_tar.addfile(tarinfo, BytesIO(file_data))
pw_tar.close()

pw_tarstream.seek(0)
nmap = cli.images.build(
    fileobj=pw_tarstream,
    custom_context=True,
    tag='hat/nmap'
)
nmap?

In [ ]:
## nmap Stack
os.chdir('/home/jovyan/work')
func_stack='''
provider:
  name: faas
  gateway: http://gateway:8080

functions:
  nmap:
    lang: Dockerfile
    handler: ./Dockerfile
    image: hat/nmap
'''
with open("nmap_func.yml", "w") as stack:
    stack.write("%s" % func_stack)

In [ ]:
!faas-cli build -f nmap_func.yml

In [ ]:
!faas-cli deploy -f nmap_func.yml

In [ ]:
## Testing
!curl -v http://gateway:8080/system/functions

In [ ]:
## Executing nmap \o/
!curl -v --data "-T4 172.17.0.3" http://gateway:8080/function/nmap

Async IO with docker
-> faas in python
https://pypi.python.org/pypi/aiodocker/0.8.2
http://aiodocker.readthedocs.io/en/latest/
https://curio.readthedocs.io/en/latest/tutorial.html

In [ ]:
import asyncio
from aiodocker.docker import Docker
from aiodocker.exceptions import DockerError


async def demo(docker):
    try:
        await docker.images.get('alpine:latest')
    except DockerError as e:
        if e.status == 404:
            await docker.pull('alpine:latest')
        else:
            print('Error retrieving alpine:latest image.')
            return

    config = {
        # "Cmd": ["/bin/ash", "-c", "sleep 1; echo a; sleep 1; echo a; sleep 1; echo a; sleep 1; echo x"],
        "Cmd": ["/bin/ash"],
        "Image": "alpine:latest",
        "AttachStdin": True,
        "AttachStdout": True,
        "AttachStderr": True,
        "Tty": False,
        "OpenStdin": True,
        "StdinOnce": True,
    }
    container = await docker.containers.create_or_replace(
        config=config, name='aiodocker-example')
    print("created and started container {}".format(container._id[:12]))

    try:
        ws = await container.websocket(stdin=True, stdout=True, stderr=True, stream=True)
        await container.start()

        async def _send():
            await asyncio.sleep(0.5)
            await ws.send_bytes(b'echo "hello world"\n')
            print("sent a shell command")

        asyncio.ensure_future(_send())
        resp = await ws.receive()
        print("received: {}".format(resp))
        await ws.close()

        output = await container.log(stdout=True)
        print("log output: {}".format(output))
    finally:
        print("removing container")
        await container.delete(force=True)


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    docker = Docker()
    try:
        loop.run_until_complete(demo(docker))
    finally:
        loop.run_until_complete(docker.close())
        loop.close()

In [ ]:
import asyncio
from aiodocker.docker import Docker


async def demo(docker):
    print('--------------------------------')
    print('- Check Docker Version Information')
    data_version = await docker.version()
    for key, value in data_version.items():
        print(key, ':', value)

    print('--------------------------------')
    print('- Check Docker Image List')
    images = await docker.images.list()
    for image in images:
        for key, value in image.items():
            if key == 'RepoTags':
                print(key, ':', value)

    print('--------------------------------')
    print('- Check Docker Container List')
    containers = await docker.containers.list()
    for container in containers:
        container_show = await container.show()
        for key, value in container_show.items():
            if key == 'Id':
                print('Id', ':', value[:12])
    print('--------------------------------')


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    docker = Docker()
    try:
        loop.run_until_complete(demo(docker))
    finally:
        loop.run_until_complete(docker.close())
        loop.close()

In [ ]:
import asyncio
from aiodocker.docker import Docker
from aiodocker.exceptions import DockerError


async def demo(docker):
    try:
        await docker.images.get('alpine:latest')
    except DockerError as e:
        if e.status == 404:
            await docker.pull('alpine:latest')
        else:
            print('Error retrieving alpine:latest image.')
            return

    subscriber = docker.events.subscribe()

    config = {
        "Cmd": ["tail", "-f", "/var/log/dmesg"],
        "Image":"alpine:latest",
         "AttachStdin": False,
         "AttachStdout": True,
         "AttachStderr": True,
         "Tty": False,
         "OpenStdin": False,
         "StdinOnce": False,
    }
    container = await docker.containers.create_or_replace(
        config=config, name='testing')
    await container.start()
    print("=> created and started container {}".format(container._id[:12]))

    while True:
        event = await subscriber.get()
        if event is None:
            break

        for key, value in event.items():
            print(key,':', value)

        # Demonstrate simple event-driven container mgmt.
        if event['Actor']['ID'] == container._id:
            if event['Action'] == 'start':
                await container.stop()
                print("=> killed {}".format(container._id[:12]))
            elif event['Action'] == 'stop':
                await container.delete(force=True)
                print("=> deleted {}".format(container._id[:12]))
            elif event['Action'] == 'destroy':
                print('=> done with this container!')
                break

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    docker = Docker()
    try:
        # do our stuffs.
        loop.run_until_complete(demo(docker))
    finally:
        loop.run_until_complete(docker.close())
        loop.close()

In [ ]:
import asyncio
import aiodocker

async def list_things():
    docker = aiodocker.Docker()
    print('== Images ==')
    for image in (await docker.images.list()):
        tags = image['RepoTags'][0] if image['RepoTags'] else ''
        print(image['Id'], tags)
    print('== Containers ==')
    for container in (await docker.containers.list()):
        print(f" {container._id}")
    await docker.close()

async def run_container():
    docker = aiodocker.Docker()
    print('== Running a hello-world container ==')
    container = await docker.containers.create_or_replace(
        config={
            'Cmd': ['/bin/ash', '-c', 'echo "hello world"'],
            'Image': 'alpine:latest',
        },
        name='testing',
    )
    await container.start()
    logs = await container.log(stdout=True)
    print(''.join(logs))
    await container.delete(force=True)
    await docker.close()

if __name__ == '__main__':
    loop2 = asyncio.get_event_loop()
    loop2.run_until_complete(list_things())
    loop2.run_until_complete(run_container())
    loop2.close()